In [ ]:
import dateutil.parser
import glob,os

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster import KMeansClusterer
from nltk.corpus import stopwords

import numpy as np
import pandas as pd

from gensim.models import Doc2Vec, Word2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.metrics import confusion_matrix

import pickle
import _pickle as cPickle

In [2]:
path = 'C:\\Users\\ab356917\\Downloads\\Cluster-data'
data = []
for infile in glob.glob( os.path.join(path, '*.*') ):
       with open(infile) as file:
            string = file.readlines()
            for each_line in string:
                if len(each_line.split())>5: # to remove time and name from the doc
                    data.extend(each_line.split(":")[2:])

In [3]:
# sentenceLabeled = []
# for sentenceID, sentence in enumerate(data):
#     sentenceL = TaggedDocument(words=sentence.split(), tags = ['SENT_%s' %sentenceID])
#     sentenceLabeled.append(sentenceL)

In [4]:
# doc2vec_model = Doc2Vec(vector_size=300, window=10, min_count=0, workers=11, alpha=0.005, min_alpha=0.025)
# doc2vec_model.build_vocab(sentenceLabeled)

In [5]:
# for epoch in range(20):
#     print(epoch ,"is running")
#     doc2vec_model.train(sentenceLabeled,total_examples=len(sent_tokenize(data)),epochs=2000)
#     doc2vec_model.alpha -= 0.0008  # decrease the learning rate
#     doc2vec_model.min_alpha = doc2vec_model.alpha  # fix the learning rate, no decay
#     doc2vec_model.save("doc2vec_model-withoutTime-g.model")
# textVect = doc2vec_model.docvecs.doctag_syn0

In [6]:
doc2vec_model = Doc2Vec.load("doc2vec_model-withoutTime-g.model")
textVect = doc2vec_model.docvecs.vectors_docs

In [191]:
## K-means ##
num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(textVect)
clusters = km.labels_.tolist()

## Print Sentence Clusters ##
cluster_info = {'sentence': data, 'cluster' : clusters}
sentenceDF = pd.DataFrame(cluster_info, index=[clusters], columns = ['sentence','cluster'])

# for num in range(num_clusters):
#     print()
#     print("Sentence cluster %d: " %int(num+1), end='')
#     print()
#     for sentence in sentenceDF.ix[num]['sentence'].values.tolist():
#         print(' %s ' %sentence, end='')
#     print()

In [8]:
# Importing files containing Ques, Ans and Others data
ques_file = open('C:\\Users\\ab356917\\Downloads\\ques.txt','r').readlines()
ans_file = open('C:\\Users\\ab356917\\Downloads\\ans.txt','r').readlines()
others_file = open('C:\\Users\\ab356917\\Downloads\\others.txt','r').readlines()

In [10]:
# List of all Concatenated Questions, Concatenated Answers, Concatenated Others
Questions_All = [" ".join([i.replace('\n','') for i in ques_file])]
Answers_All = [" ".join([i.replace('\n','') for i in ans_file])]
Others_All = [" ".join([i.replace('\n','') for i in others_file])]

# List of ques, ans and others
ques_list = [i.replace('\n','') for i in ques_file]
ans_list = [i.replace('\n','') for i in ans_file]
others_list = [i.replace('\n','') for i in others_file]

# List containing All concatenated Questions, list of Answers and List of Others
AllQuestion_ans_others = Questions_All + ans_list + others_list
AllAnswer_ques_others = Answers_All + ques_list + others_list
AllOthers_ans_ques = Others_All + ans_list + ques_list

In [12]:
stop_words = ['i','me', 'my', 'myself','we','our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their','theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being','have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing','a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for','with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'all','any', 'both', 'each', 'few','more', 'most', 'other','some','such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will','just', 'don', 'should', 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn', 'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn','weren','won','wouldn']

# Questions features

In [13]:
#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = tfidf_vectorizer.fit_transform(AllQuestion_ans_others).todense() #fit the vectorizer to synopses

In [14]:
ques_features = list(zip(tfidf_vectorizer.get_feature_names(), tfidf_matrix.tolist()[0]))
ques_features = sorted(ques_features, key=lambda x: x[1], reverse= True)
ques_features = [i for i in ques_features if i[1]>0]

# Answers features

In [15]:
#define vectorizer parameters
tfidf_vectorizer_ans = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_ans = tfidf_vectorizer_ans.fit_transform(AllAnswer_ques_others).todense() #fit the vectorizer to synopses

In [16]:
ans_features = list(zip(tfidf_vectorizer_ans.get_feature_names(), tfidf_matrix_ans.tolist()[0]))
ans_features = sorted(ans_features, key=lambda x: x[1], reverse= True)
ans_features = [i for i in ans_features if i[1]>0]

# Others feature

In [17]:
#define vectorizer parameters
tfidf_vectorizer_others = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix_others = tfidf_vectorizer_others.fit_transform(AllOthers_ans_ques).todense() #fit the vectorizer to synopses

In [18]:
others_features = list(zip(tfidf_vectorizer_others.get_feature_names(), tfidf_matrix_others.tolist()[0]))
others_features = sorted(others_features, key=lambda x: x[1], reverse= True)
others_features = [i for i in others_features if i[1]>0]

In [312]:
all_features = [i[0] for i in ques_features] + [i[0] for i in ans_features] + [i[0] for i in others_features]
all_features = list(set(all_features))
all_features = [x for x in all_features if not x.strip().isdigit() ]

# Frequency Matrix

In [22]:
count_vectorizer = CountVectorizer()
count_vectorizer.fit_transform(all_features)
#print("Vocabulary:", count_vectorizer.vocabulary_)

<435x435 sparse matrix of type '<class 'numpy.int64'>'
	with 435 stored elements in Compressed Sparse Row format>

In [184]:
def train_test(ques_list,ans_list,others_list,test_size = 0.4):
    train_ques,test_ques = train_test_split(ques_list,test_size=test_size)
    ques_train_class_list, ques_test_class_list = train_test_split(["Ques"]*len(ques_list),test_size=test_size)
    train_ans,test_ans = train_test_split(ans_list,test_size=test_size)
    ans_train_class_list, ans_test_class_list = train_test_split(["Ans"]*len(ans_list),test_size=test_size)
    train_other,test_other = train_test_split(others_list,test_size=test_size)
    other_train_class_list, other_test_class_list = train_test_split(["Others"]*len(others_list),test_size=test_size)
    
    train_data = train_ques + train_ans + train_other
    test_data =  test_ques + test_ans + test_other
    
    class_list_train = ques_train_class_list + ans_train_class_list + other_train_class_list 
    class_list_test = ques_test_class_list + ans_test_class_list + other_test_class_list
    return train_data, test_data, class_list_train, class_list_test, test_ques, test_ans, test_other # Modified

In [335]:
max_features = ['auto',50,75,100,150] 
max_depth = [10,20,30,50]
max_leaf_nodes = [3,7,9,12]

In [301]:
def create_model(train_data, class_list_train,max_features,max_depth,max_leaf_nodes):
    freq_term_matrix = count_vectorizer.transform(train_data)
    rf = RandomForestClassifier(n_estimators=100, max_features=max_features,max_depth=max_depth, max_leaf_nodes=max_leaf_nodes)
    rf.fit(freq_term_matrix.todense(),class_list_train)
    return rf

In [294]:
def get_accuracy_from_confusion_matrix(test_data,class_list_test):
    cm = confusion_matrix(class_list_test, rf.predict(count_vectorizer.transform(test_data)),labels=class_list_test)
    accuracy = np.diagonal(cm).sum()/len(class_list_test)
    return accuracy

In [186]:
# for i in range(50):
#     train_data, test_data, class_list_train, class_list_test, test_ques, test_ans, test_other = train_test(ques_list,ans_list,others_list,0.25)
#     rf = create_model(train_data, class_list_train)
#     Train_accuracy = accuracy_score(class_list_train, rf.predict(count_vectorizer.transform(train_data)))
#     Test_accuracy = accuracy_score(class_list_test, rf.predict(count_vectorizer.transform(test_data)))
#     Ques_accuracy = accuracy_score(["Ques"]*len(test_ques), rf.predict(count_vectorizer.transform(test_ques))) # Modified
#     Ans_accuracy = accuracy_score(["Ans"]*len(test_ans), rf.predict(count_vectorizer.transform(test_ans))) # Modified
#     Others_accuracy = accuracy_score(["Others"]*len(test_other), rf.predict(count_vectorizer.transform(test_other))) # Modified
#     with open('C:\\Users\\ab356917\\Downloads\\rf-models\\rf{0}_TrainAcc_{1}_TestAcc_{2}'.format(i,round(Train_accuracy,2),round(Test_accuracy,2)), 'wb') as f:
#         cPickle.dump(rf, f)
#     print("For iter {0} Training Accuracy is: {1} and Test Accuracy is : {2}".format(i,Train_accuracy,Test_accuracy))
#     print("             Question Accuracy is: {0}, Answer Accuracy is : {1} and Others Accuracy is : {2}".format(round(Ques_accuracy,4),round(Ans_accuracy,4), round(Others_accuracy,4)))
#     print()

In [299]:
train_data, test_data, class_list_train, class_list_test, test_ques, test_ans, test_other = train_test(ques_list,ans_list,others_list,0.4)

In [343]:
list_df = []
for i in max_features:
    for j in max_depth:
        for k in max_leaf_nodes:
            rf = create_model(train_data, class_list_train,i,j,k)
            cv = cross_validation.KFold(len(train_data), n_folds=10, random_state=3)
            results = cross_validation.cross_val_score(rf, count_vectorizer.transform(train_data), rf.predict(count_vectorizer.transform(train_data)), cv=cv)
            Accuracy = get_accuracy_from_confusion_matrix(test_data,class_list_test)
            list_df.append((i, j, k, np.mean(results), np.max(results) - np.min(results), Accuracy))
df = pd.DataFrame.from_records(list_df, columns=["Max_fatures","max_depth","max_leaf_nodes","Train-Accuracy(mean)","Train-Accuracy(max - min)","Test-Accuracy"])

In [387]:
#cf.sort_values(by=['Train-Accuracy(max - min)','Test-Accuracy'], ascending=[True,False]).head()
df = df.sort_values(by=['Train-Accuracy(mean)','Test-Accuracy','Train-Accuracy(max - min)'], ascending=[False,True,True])
df.head(1)

Max_fatures  max_depth  max_leaf_nodes  Train-Accuracy(mean)  \
76         150         50               3              0.974211   

    Train-Accuracy(max - min)  Test-Accuracy  
76                   0.105263       0.545455

In [380]:
rfc1 = create_model(train_data, class_list_train,max_features=150,max_depth=50, max_leaf_nodes=3)

In [381]:
for i, sentence in enumerate(test_data):
    if(rfc1.predict(count_vectorizer.transform([sentence])) == class_list_test[i]):
        print(sentence," ::" ,rfc1.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

Not good my new computer is not working  :: ['Ques'] :: Ques
Do you know which application it was?  :: ['Ques'] :: Ques
just a second  :: ['Ques'] :: Ques
Did it took that long during boot up ?  :: ['Ques'] :: Ques
It didn't do this until i installed updates  :: ['Ques'] :: Ques
May I know if you are chatting from the same system that has the issue or from a different system?  :: ['Ques'] :: Ques
May I know do you have another device like mobile or tablet?  :: ['Ques'] :: Ques
No we will check the hardware part is fine or not  :: ['Ques'] :: Ques
Please allow me a few minutes while I go ahead and do a quick research on this issue.  :: ['Ques'] :: Ques
I apologize for the inconvenience. I will do my best to help you.  :: ['Ques'] :: Ques
Yeah its on. Im running a system check  :: ['Ques'] :: Ques
OK. But it never did this before.  :: ['Ques'] :: Ques
Please give me a chance I will definitely try my best to resolve the issue for you.  :: ['Ques'] :: Ques
It definitely booted up faster th

In [382]:
for i, sentence in enumerate(test_data):
    if(rfc1.predict(count_vectorizer.transform([sentence])) != class_list_test[i]):
        print(sentence," ::" ,rfc1.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

It was recently updated too.  No my phone   :: ['Ans'] :: Ques
Please provide a phone number to contact you back, in case we lost the connection.  :: ['Ans'] :: Ques
Please wait  :: ['Ques'] :: Ans
Could you please provide me your contact number in case the chat gets disconnected, we can call you back to resume the troubleshooting?  :: ['Ques'] :: Ans
control.app05-04.logmeinrescue-enterprise.com (63.251.34.136  443)  :: ['Others'] :: Ans
Please provide type www.hp.com/123 in phone  :: ['Others'] :: Ans
I am fine. Thank you for asking.  :: ['Ques'] :: Ans
Please enter this code  :: ['Ques'] :: Ans
Please type www.hp.com/123 in the phone now  :: ['Others'] :: Ans
yes you have  :: ['Ques'] :: Ans
An automatic update, updated my computer last week and now the computer is running very slowly and takes a long time to boot up after I first log on  :: ['Ques'] :: Ans
control.app04-07.logmeinrescue-enterprise.com (64.74.103.155  443)  :: ['Others'] :: Ans
May I know the progress?  :: ['Ques'] 

In [244]:
def model_Prediction(n_folds, random_state):
    rf = create_model(train_data, class_list_train)
    cv = cross_validation.KFold(len(train_data), n_folds=n_folds, random_state=random_state)
    results = cross_validation.cross_val_score(rf, count_vectorizer.transform(train_data), rf.predict(count_vectorizer.transform(train_data)), cv=cv)
    return np.mean(results) , np.var(results), results

In [292]:
# # TRAINING ACCURACY TO DECIDE FOLD AND SEED IN CROSS-VALIDATION
#n_folds = [10,15,20,25]
#random_state = range(1,11)
# for i in random_state:
#     for j in n_folds:
#         mean,var,results = model_Prediction(j,i)
#         diff = max(results)-min(results)
#         print("seed {0} and {1}-fold Mean Accuracy is: {2} with Var: {3} & max-min = {4}".format(i,j,mean,var,diff))
#     print()

## Grid Search-SKLEARN to select HyperParameter

In [361]:
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

C:\Users\ab356917\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [377]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators' : [50,100,150,200,250,300,500] ,'max_features' : ['auto',50,75,100,150] ,'max_depth' : [10,20,30,50],'max_leaf_nodes' : [3,7,9,12]}]

In [378]:
rfc = RandomForestClassifier(n_estimators=100, max_features='auto',max_depth=10, max_leaf_nodes=3)

In [379]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=tuned_parameters, cv= 5)
CV_rfc.fit(count_vectorizer.transform(train_data), class_list_train)
print(CV_rfc.best_params_)

{'max_depth': 50, 'max_features': 150, 'max_leaf_nodes': 9, 'n_estimators': 100}


In [383]:
from sklearn.metrics import classification_report

In [390]:
y_true, y_pred = class_list_test, CV_rfc.predict(count_vectorizer.transform(test_data))
print(classification_report(y_true, y_pred))
print()

             precision    recall  f1-score   support

        Ans       0.35      0.16      0.22        38
     Others       0.52      0.37      0.43        38
       Ques       0.56      0.88      0.68        56

avg / total       0.49      0.52      0.48       132




In [393]:
for i, sentence in enumerate(test_data):
    if(CV_rfc.predict(count_vectorizer.transform([sentence])) == class_list_test[i]):
        print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

Do you know which application it was?  :: ['Ques'] :: Ques
just a second  :: ['Ques'] :: Ques
Did it took that long during boot up ?  :: ['Ques'] :: Ques
It didn't do this until i installed updates  :: ['Ques'] :: Ques
May I know if you are chatting from the same system that has the issue or from a different system?  :: ['Ques'] :: Ques
May I know do you have another device like mobile or tablet?  :: ['Ques'] :: Ques
No we will check the hardware part is fine or not  :: ['Ques'] :: Ques
Please allow me a few minutes while I go ahead and do a quick research on this issue.  :: ['Ques'] :: Ques
I apologize for the inconvenience. I will do my best to help you.  :: ['Ques'] :: Ques
Yeah its on. Im running a system check  :: ['Ques'] :: Ques
OK. But it never did this before.  :: ['Ques'] :: Ques
Please give me a chance I will definitely try my best to resolve the issue for you.  :: ['Ques'] :: Ques
It definitely booted up faster this time. What update caused the issue?  :: ['Ques'] :: Ques
Y

In [394]:
for i, sentence in enumerate(test_data):
    if(CV_rfc.predict(count_vectorizer.transform([sentence])) != class_list_test[i]):
        print(sentence," ::" ,CV_rfc.predict(count_vectorizer.transform([sentence])),"::", class_list_test[i])

Not good my new computer is not working  :: ['Ans'] :: Ques
It was recently updated too.  No my phone   :: ['Ans'] :: Ques
Please provide a phone number to contact you back, in case we lost the connection.  :: ['Ans'] :: Ques
I dont see file. I see lock,switch user,sign out,change password and task mangager  :: ['Ans'] :: Ques
A new window will open.  :: ['Ans'] :: Ques
It is at the page where i enter my password  :: ['Others'] :: Ques
Once the task bar open then you will get an option as "File".  :: ['Ans'] :: Ques
Please wait  :: ['Ques'] :: Ans
Could you please provide me your contact number in case the chat gets disconnected, we can call you back to resume the troubleshooting?  :: ['Ques'] :: Ans
control.app05-04.logmeinrescue-enterprise.com (63.251.34.136  443)  :: ['Others'] :: Ans
Please provide type www.hp.com/123 in phone  :: ['Others'] :: Ans
I am fine. Thank you for asking.  :: ['Ques'] :: Ans
Please enter this code  :: ['Others'] :: Ans
Please type www.hp.com/123 in the pho

In [42]:
def get_class_features(list_of_sentences, count_vectorizer = count_vectorizer, ordered_features = ordered_features, rf = rf ):
    
    class_of = count_vectorizer.transform(list_of_sentences)
    class_is = rf.predict(class_of)
    
    features = []
    for each_class in class_of.todense().tolist():
        test = list(zip(ordered_features, each_class))
        features.append([i for i in test if i[1]>0])
    
    class_feature = {}
    for index, each_sentence in enumerate(list_of_sentences):
        class_feature.update({each_sentence:{"class": class_is[index], "feature": features[index]}})
    return class_feature

In [43]:
input_document = ["Do i ask check the create this task with administrative privileges?", "My laptop is not working.","How are you?","While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy."]

In [44]:
test = list(zip(ordered_features, class_of.todense().tolist()[0]))

In [45]:
get_class_features(input_document)

{'Do i ask check the create this task with administrative privileges?': {'class': 'Ques',
  'feature': [('administrative', 1),
   ('ask', 1),
   ('check', 1),
   ('create', 1),
   ('privileges', 1),
   ('task', 1)]},
 'How are you?': {'class': 'Others', 'feature': [('how', 1)]},
 'My laptop is not working.': {'class': 'Ques',
  'feature': [('laptop', 1), ('working', 1)]},
 'While troubleshoot I checked the device drivers and software and let me tell you, your HP laptop is just fit and dandy.': {'class': 'Ans',
  'feature': [('checked', 1),
   ('dandy', 1),
   ('device', 1),
   ('drivers', 1),
   ('fit', 1),
   ('hp', 1),
   ('laptop', 1),
   ('let', 1),
   ('software', 1),
   ('tell', 1),
   ('troubleshoot', 1)]}}

In [315]:
print("Train Accuracy :: ", accuracy_score(class_list, rf.predict(count_vectorizer.transform(train_ques+train_ans+train_other))))

Train Accuracy ::  0.69262295082


In [50]:
print("Test Accuracy :: ", accuracy_score(ques_test_class_list+ans_test_class_list+other_test_class_list, rf.predict(count_vectorizer.transform(test_ques+test_ans+test_other))))

Test Accuracy ::  0.506024096386
